> ## Assignment-1 Supervised Learning


## Exploratory Analysis
To begin this exploratory analysis, first import libraries and define functions for plotting the data using `matplotlib`. Depending on the data, not all plots will be made. (Hey, I'm just a simple kerneling bot, not a Kaggle Competitions Grandmaster!)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


There is 1 csv file in the current version of the dataset:


In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


The next hidden code cells define functions for plotting data. Click on the "Code" button in the published kernel to reveal the hidden code.

### Let's check 1st file: /kaggle/input/data.csv

In [ ]:
nRowsRead = None # specify 'None' if want to read whole file
# data.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df = pd.read_csv('/kaggle/input/data.csv', delimiter=',', nrows = nRowsRead)
df.dataframeName = 'data.csv'
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')

Let's take a quick look at what the data looks like:

In [ ]:
df.describe()

In [ ]:
# Checking labels distributions
import seaborn as sns
sns.set(context = 'paper')

plt.figure(figsize = (10,5))
sns.countplot(df['Bankrupt?'])
plt.title('Class Distributions \n (Bankrupt?:FALSE||Bankrupt?:TRUE)', fontsize=14)
plt.show()


In [ ]:
#split features and classification
X = df.drop('Bankrupt?', axis = 1)       
y = df['Bankrupt?']


In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
#since we are dealing with imbalanced data, we need to apply the synthetic minority oversampling technique 
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X, y = smote.fit_resample(X, y)


In [ ]:
#split train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# Data structures for storing best accuracies, training time and test time
num_classifiers = 5
best_accuracy = np.zeros(num_classifiers)
train_time = np.zeros(num_classifiers)
test_time = np.zeros(num_classifiers)

# Algorithm-1 Decision Trees

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import KFold

In [ ]:
clf_dt = tree.DecisionTreeClassifier(random_state=42)
clf_dt.fit(X_train, y_train)
y_pred = clf_dt.predict(X_test)
dt_accuracy = accuracy_score(y_test, y_pred)
print(dt_accuracy)

In [ ]:
fig_path = '/kaggle/output'

In [ ]:
depth_range = np.arange(20) + 1
train_scores, test_scores = validation_curve(clf_dt, X_train, y_train, param_name="max_depth", param_range=depth_range, cv=5)

plt.figure()
plt.xticks(depth_range)
plt.plot(depth_range, np.mean(train_scores, axis=1), label='Training score')
plt.plot(depth_range, np.mean(test_scores, axis=1), label='Cross-validation score')
plt.xlabel('Max Depth')
plt.ylabel("Score")
plt.legend(loc="best")
plt.grid()
plt.savefig(fig_path + 'dt_validation_curve_1.png')
plt.show()

#  **Hyperparameter Tuning**

In [ ]:
import time

clf_dt = tree.DecisionTreeClassifier(random_state=42)
clf_dt.fit(X_train, y_train)

depth_range = np.arange(10) + 1
tuned_params = {'max_depth' : depth_range}
clf_dt = GridSearchCV(clf_dt, param_grid=tuned_params, cv=5)
t0 = time.time()
clf_dt.fit(X_train, y_train)
t1 = time.time()
train_time[0] = t1 - t0
print('Completed training in %f seconds' % train_time[0])
best_clf_dt = clf_dt
best_dt_params = clf_dt.best_params_
print("Best parameters set for decision tree found on development set:")
print(best_dt_params)
t0 = time.time()
y_pred = clf_dt.predict(X_test)
t1 = time.time()
test_time[0] = t1 - t0
print('Inference time on test data: %f seconds' % test_time[0])
best_accuracy[0] = accuracy_score(y_test, y_pred)
print('Accuracy of decision tree is %.2f%%' % (best_accuracy[0] * 100))

In [ ]:
train_sizes = np.linspace(0.1, 1.0, 10)
_, train_scores, test_scores = learning_curve(best_clf_dt, X_train, y_train, train_sizes=train_sizes, cv=5)
plt.figure()
plt.plot(train_sizes, np.mean(train_scores, axis=1), 'o-', label='Decisin Tree Training score')
plt.plot(train_sizes, np.mean(test_scores, axis=1), 'o-', label='Decision Tree CV score')
plt.xlabel('% Training examples')
plt.ylabel("Score")
plt.legend(loc="best")
plt.grid()
plt.savefig(fig_path + 'dt_learning_curve.png')
plt.show()


In [ ]:
print(classification_report(y_test,y_pred))

# **Algorithm-2 Neural Network**

I start Neural Network analysis with feed-forward neural network by using PyTorch.

# **Preparing the data**

In [ ]:
from sklearn.neural_network import MLPClassifier


In [ ]:
clf_nn = MLPClassifier(hidden_layer_sizes=(5, 2), random_state=42, max_iter=1000)
clf_nn.fit(X_train, y_train)
y_pred = clf_nn.predict(X_test)
nn_accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of neural network without hyperparameter tuning is %.2f%%' % (nn_accuracy * 100))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_svmlight_file
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import roc_curve, auc

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6250)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=6250)

y_train=np.array(y_train)
y_valid=np.array(y_valid)
y_test=np.array(y_test)

print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
scaler = MaxAbsScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
X_valid_transformed = scaler.transform(X_valid)
X_test_transformed = scaler.transform(X_test)

print("Train - Min: {}, Max: {}".format(np.min(X_train_transformed), np.max(X_train_transformed)))
print("Valid - Min: {}, Max: {}".format(np.min(X_valid_transformed), np.max(X_train_transformed)))
print("Test  - Min: {}, Max: {}".format(np.min(X_test_transformed), np.max(X_test_transformed)))

Now, we will train a feed-forward neural network by using PyTorch. We will do the following steps in order:

1. Load the training and test datasets using DataLoader
2. Define a Feedforwad Neural Network
3. Define a loss function
4. Train the network on the training data
5. Test the network on the test data

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# lets fix the random seeds for reproducibility.
torch.manual_seed(6250)
if torch.cuda.is_available():
    torch.cuda.manual_seed(6250)

trainset = TensorDataset(torch.from_numpy(X_train_transformed.astype('float32')), torch.from_numpy(y_train.astype('float32')).view(-1,1))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

validset = TensorDataset(torch.from_numpy(X_valid_transformed.astype('float32')), torch.from_numpy(y_valid.astype('float32')).view(-1,1))
validloader = torch.utils.data.DataLoader(validset, batch_size=4, shuffle=True, num_workers=2)

testset = TensorDataset(torch.from_numpy(X_test_transformed.astype('float32')), torch.from_numpy(y_test.astype('float32')).view(-1,1))
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

In [ ]:
# from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class FeedForwardNet(nn.Module):
    def __init__(self, n_input, n_hidden, n_output):
        super(FeedForwardNet, self).__init__()
        self.hidden1 = nn.Linear(n_input, n_hidden)
        self.hidden2 = nn.Linear(n_hidden, n_hidden)
        self.out = nn.Linear(n_hidden, n_output)

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = self.out(x)
        return x

net = FeedForwardNet(n_input=95, n_hidden=5, n_output=1)

In [ ]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [ ]:
y_true = []
y_scores = []
for data in testloader:
    inputs, labels = data
    outputs = net(inputs)
    outputs = torch.sigmoid(outputs)  # since we have no activation at the end of output layer
    y_true.extend(labels.numpy().flatten().tolist())
    y_scores.extend(outputs.data.numpy().flatten().tolist())
    
fpr, tpr, _ = roc_curve(y_true, y_scores)
auc_ffnet = auc(fpr, tpr)
auc_ffnet

In [ ]:
train_losses = []
valid_losses = []
model = net

for epoch in range(10):  # loop over the dataset multiple times

    # set the model as train mode
    model.train()
    train_loss = 0.0
    train_counter = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, targets = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        train_loss += (loss.item() * inputs.size(0))
        train_counter += inputs.size(0)

    train_losses.append(train_loss/train_counter)
    
    # switch to evaluation mode
    model.eval()
    valid_loss = 0.0
    valid_counter = 0
    with torch.no_grad():
        for i, data in enumerate(validloader, 0):
            # get the inputs
            inputs, targets = data

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            valid_loss += (loss.item() * inputs.size(0))
            valid_counter += inputs.size(0)
    
    valid_losses.append(valid_loss/valid_counter)
    
print('Finished Training')

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

plt.figure()
plt.plot(np.arange(len(train_losses)), train_losses, label='Train')
plt.plot(np.arange(len(valid_losses)), valid_losses, label='Validation')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(loc="best")

In [ ]:
y_true = []
y_scores = []
for data in testloader:
    inputs, labels = data
    outputs = model(inputs)
    # outputs = torch.sigmoid(outputs) 
    y_true.extend(labels.numpy().flatten().tolist())
    y_scores.extend(outputs.data.numpy().flatten().tolist())

fprnn, tprnn, _ = roc_curve(y_true, y_scores)
auc_ffnet = auc(fprnn, tprnn)
auc_ffnet

In [ ]:
from sklearn.metrics import *

def classification_metrics_score(Y_true, Y_score, a):
    #NOTE: It is important to provide the output in the same order
    Y_score[Y_score>=a] = 1
    Y_score[Y_score<a] = 0
    Y_pred = Y_score
    acc = accuracy_score(Y_true,Y_pred)
    # bacc = balanced_accuracy_score(Y_true, Y_pred)
    auc_ = roc_auc_score(Y_true,Y_score)
    precision = precision_score(Y_true,Y_pred)
    recall = recall_score(Y_true,Y_pred)
    f1score = f1_score(Y_true,Y_pred)
    cm = confusion_matrix(Y_true,Y_pred)
    false_positive = cm[1,0]/cm.sum()

    return acc, auc_, precision, recall, f1score, cm, false_positive

In [ ]:
classification_metrics_score(np.array(y_true), np.array(y_scores),0.5)


In [ ]:
def plot_confusion_matrix(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true, y_pred)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] # Normalize
    cmap = plt.cm.Blues

    # https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]), yticks=np.arange(cm.shape[0]),
           xticklabels=class_names, yticklabels=class_names,
           title="Normalized Confusion Matrix",
           ylabel='True', xlabel='Predicted')

    # Rotate the tick labels and set alignment
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    fmt = '.2f'
    thresh = cm.max() / 2.
    # Loop over data dimensions and create text annotations
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()

    plt.show()

In [ ]:
y_true = []
y_scores = []
for data in trainloader:
    inputs, labels = data
    outputs = model(inputs)
    # outputs = torch.sigmoid(outputs) 
    y_true.extend(labels.numpy().flatten().tolist())
    y_scores.extend(outputs.data.numpy().flatten().tolist())

fprnn, tprnn, _ = roc_curve(y_true, y_scores)
auc_ffnet = auc(fprnn, tprnn)
auc_ffnet

classification_metrics_score(np.array(y_true), np.array(y_scores),0.5)


In [ ]:
plt.figure(figsize=(7,6))
lw = 2
plt.plot(fprnn, tprnn, color='blue',lw=lw, label='MLP (AUC = %0.2f)' % auc_ffnet)
plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi-layer Perceptron')
plt.legend(loc="lower right")
plt.show()


In [ ]:
y_pred = np.array(y_scores.copy())
y_pred[y_pred>=0.5] = 1
y_pred[y_pred<0.5] = 0


In [ ]:
plot_confusion_matrix(y_true, y_pred, ["Bankruptcy:False","Bankruptcy:True"])

# **Algorithm-3 Boosting**

We use AdaBoostClassifier with decision tree stumps.

We are now pruning hyperparameters like the number of weak learners and anayze how it affects the performance.

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
#split train and test data (revert from MLP data)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
dt_stump = tree.DecisionTreeClassifier(max_depth=1, min_samples_leaf=1)
clf_boosted = AdaBoostClassifier(base_estimator=dt_stump, random_state=7)
clf_boosted.fit(X_train, y_train)
y_pred = clf_boosted.predict(X_test)
boosted_accuracy = accuracy_score(y_test, y_pred)
print('Accuracy of Adaboost without hyperparameter tuning is %.2f%%' % (boosted_accuracy * 100))


In [ ]:
#Define AdaBoost learner
num_learners = 500
dt_stump = tree.DecisionTreeClassifier(max_depth=1, min_samples_leaf=1)
clf_boosted = AdaBoostClassifier(base_estimator=dt_stump, n_estimators=num_learners, random_state=42)

# Cross-validation
num_folds = 5
kf = KFold(n_splits=num_folds, random_state=7)
train_scores = np.zeros((num_learners, num_folds))
val_scores = np.zeros((num_learners, num_folds))
for idx, (train_index, test_index) in enumerate(kf.split(X_train)):
    clf_boosted.fit(X_train[train_index], y_train[train_index])
    train_scores[:, idx] = np.asarray(list(clf_boosted.staged_score(X_train[train_index], y_train[train_index])))
    val_scores[:, idx] = np.asarray(list(clf_boosted.staged_score(X_train[test_index], y_train[test_index])))

n_estimators_range = np.arange(num_learners) + 1
plt.figure(figsize=(7,6))
plt.plot(n_estimators_range, np.mean(train_scores, axis=1), label='Training score')
plt.plot(n_estimators_range, np.mean(val_scores, axis=1), label='Cross-validation score')
plt.xlabel('Number of weak learners')
plt.ylabel("Score")
plt.legend(loc="best")
plt.grid()
plt.show()

In [ ]:
num_learners_optimal = np.argmax(np.mean(val_scores, axis=1)) + 1
print('Optimal number of learners for AdaBoost: %d' % num_learners_optimal)
best_clf_boosted = AdaBoostClassifier(base_estimator=dt_stump, n_estimators=num_learners_optimal, random_state=7)
t0 = time.time()
best_clf_boosted.fit(X_train, y_train)
t1 = time.time()
train_time[2] = t1 - t0
print('Completed training in %f seconds' % train_time[2])
t0 = time.time()
y_pred = best_clf_boosted.predict(X_test)
t1 = time.time()
test_time[0] = t1 - t0
print('Inference time on test data: %f seconds' % test_time[2])
best_accuracy[0] = accuracy_score(y_test, y_pred)
print('Accuracy of Adaboost with the best hyperparameters is %.2f%%' % (best_accuracy[0] * 100))

In [ ]:
train_sizes = np.linspace(0.1, 1.0, 5)
best_clf_boosted = AdaBoostClassifier(base_estimator=dt_stump, n_estimators=num_learners_optimal, random_state=7)
_, train_scores, test_scores = learning_curve(best_clf_boosted, X_train, y_train, train_sizes=train_sizes, cv=5)

plt.figure()
plt.plot(train_sizes, np.mean(train_scores, axis=1), 'o-', label='Training score')
plt.plot(train_sizes, np.mean(test_scores, axis=1), 'o-', label='Cross-validation score')
plt.xlabel('% training examples')
plt.ylabel("Score")
plt.legend(loc="best")
plt.grid()
#plt.savefig(fig_path + 'dt_learning_curve.png')
plt.show()

In [ ]:
plot_confusion_matrix(y_test, y_pred, ["Bankruptcy:False","Bankruptcy:True"])

# **Algorithm-4 SVM**

In [ ]:
scaler = MaxAbsScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn import svm

svm_linear = svm.LinearSVC()
svm_linear.fit(X_train, y_train)


In [ ]:
C_range = np.logspace(-3, 3, 7)
train_scores, test_scores = validation_curve(svm_linear, X_train, y_train, param_name="C", param_range=C_range, cv=5)

plt.figure()
plt.semilogx(C_range, np.mean(train_scores, axis=1), label='Training score')
plt.semilogx(C_range, np.mean(test_scores, axis=1), label='Score')
plt.xlabel('C')
plt.ylabel("Classification score")
plt.legend(loc="best")
plt.grid()
#plt.savefig(fig_path + 'dt_validation_curve_1.png')
plt.show()

In [ ]:
C_range = np.logspace(-2, 1, 10)
tuned_params = {'C' : C_range}
svm_linear = GridSearchCV(svm_linear, param_grid=tuned_params, cv=5)
t0 = time.time()
svm_linear.fit(X_train, y_train)
t1 = time.time()
train_time[3] = t1 - t0
print('Completed training in %f seconds' % train_time[3])
best_clf_svm = svm_linear
best_params = svm_linear.best_params_
print("Best parameters set found on development set:")
print(best_params)
t0 = time.time()
y_pred = best_clf_svm.predict(X_test)
t1 = time.time()
test_time[3] = t1 - t0
print('Inference time on test data: %f seconds' % test_time[3])
best_accuracy[3] = accuracy_score(y_test, y_pred)
print('Best accuracy with SVM (linear kernel) is %.2f%%' % (best_accuracy[3] * 100))

In [ ]:
train_sizes = np.linspace(0.1, 1.0, 5)
_, train_scores, test_scores = learning_curve(best_clf_svm, X_train, y_train, train_sizes=train_sizes, cv=5)

plt.figure()
plt.plot(train_sizes, np.mean(train_scores, axis=1), 'o-', label='Training score')
plt.plot(train_sizes, np.mean(test_scores, axis=1), 'o-', label='Cross-validation score')
plt.xlabel('% training examples')
plt.ylabel("Score")
plt.legend(loc="best")
plt.grid()
#plt.savefig(fig_path + 'dt_learning_curve.png')
plt.show()

In [ ]:
plot_confusion_matrix(y_test, y_pred, ["Bankruptcy:False","Bankruptcy:True"])

# **Algorithm-5 kNN**

In [ ]:
k_range = np.arange(1, 15)
train_scores, test_scores = validation_curve(KNeighborsClassifier(), X_train, y_train, param_name="n_neighbors", 
                                             param_range=k_range, cv=5)

plt.figure()
plt.plot(k_range, np.mean(train_scores, axis=1), label='Training score')
plt.plot(k_range, np.mean(test_scores, axis=1), label='Cross-validation score')
plt.xlabel('k')
plt.ylabel("Score")
plt.legend(loc="best")
plt.grid()
#plt.savefig(fig_path + 'dt_validation_curve_1.png')
plt.show()


In [ ]:
import time 

k_optimal = np.argmax(np.mean(test_scores, axis=1)) + 1
print('Optimal value of k: %d' % k_optimal)
best_clf_knn = KNeighborsClassifier(n_neighbors=k_optimal)
t0 = time.time()
best_clf_knn.fit(X_train, y_train)
t1 = time.time()
train_time[4] = t1 - t0
print('Completed training in %f seconds' % train_time[4])
t0 = time.time()
y_pred = best_clf_knn.predict(X_test)
t1 = time.time()
test_time[4] = t1 - t0
print('Inference time on test data: %f seconds' % test_time[4])
best_accuracy[4] = accuracy_score(y_test, y_pred)
print('Accuracy of kNN with k = %d is %.2f%%' % (k_optimal, best_accuracy[4] * 100))

In [ ]:
train_sizes = np.linspace(0.1, 1.0, 5)
_, train_scores, test_scores = learning_curve(best_clf_knn, X_train, y_train, train_sizes=train_sizes, cv=5)

plt.figure()
plt.plot(train_sizes, np.mean(train_scores, axis=1), 'o-', label='Training score')
plt.plot(train_sizes, np.mean(test_scores, axis=1), 'o-', label='Cross-validation score')
plt.xlabel('% training examples')
plt.ylabel("Score")
plt.legend(loc="best")
plt.grid()
#plt.savefig(fig_path + 'dt_learning_curve.png')
plt.show()

In [ ]:
plot_confusion_matrix(y_test, y_pred, ["Bankruptcy:False","Bankruptcy:True"])

## Conclusion
This concludes your starter analysis! To go forward from here, click the blue "Fork Notebook" button at the top of this kernel. This will create a copy of the code and environment for you to edit. Delete, modify, and add code as you please. Happy Kaggling!